### Import

In [23]:
import requests
import pymysql
import json
import pandas as pd
import datetime as dt
import pickle
from bs4 import BeautifulSoup as bs

### DB 사용 예시

In [44]:
con = pymysql.connect(host='localhost', user='ssafy', password='ssafy', 
                      db='access_db', charset='utf8mb4', autocommit=True)
cur = con.cursor()

sql = "INSERT INTO table_name (value1, value2) VALUES (123, '이름');"
cur.execute(sql)

con.commit()
con.close()


### 환율

##### api 함수

In [5]:
def exchange_rate_api(date):
  API_HOST = "https://www.koreaexim.go.kr/site/program/financial/exchangeJSON"
  key = "?authkey=b60MyMtLUDVQyyiDjZRdFGbKxoJBypkl"
  search = "&searchdate="+date
  data = "&data=AP01"
  
  url = API_HOST+key+search+data
  headers = {"Content-Type" : "application/json", "charset" : "UTF-8", "Accept":"*/*"}

  try:
    response = requests.get(url, headers=headers)
    return response
  except Exception as ex:
    print(ex)
  

##### 데이터 수집

In [6]:
date_range = pd.date_range(start='1/1/2011', end='12/31/2022')

result_list = []
for date in date_range:
  if dt.date.weekday(date) in (5, 6) :
    continue
  str_date = str(date).split(" ")[0]
  
  r = json.loads(exchange_rate_api(str_date).text)
  
  flag = True
  for data in r:
    if data["result"] == 4:
      flag = False
      break
    if data["cur_nm"] in ('미국 달러', '일본 옌', '유로') :
      if(data["cur_nm"] == '일본 옌'):
        result_list.append("('{}', '{}', {})".format(str_date, '일본 100엔', data["deal_bas_r"].replace(",", "")))
      else
        result_list.append("('{}', '{}', {})".format(str_date, data['cur_nm'], data["deal_bas_r"].replace(",", "")))
  if flag == False:
    break

##### 저장 및 불러오기

In [ ]:
with open("data.p", "wb") as f:
  pickle.dump(result_list, f)

In [5]:
with open("data.p", "rb") as f:
  result_list = pickle.load(f)

##### DB 저장

In [14]:
con = pymysql.connect(host='localhost', user='ssafy', password='ssafy', 
                      db='access_db', charset='utf8mb4', autocommit=True)
cur = con.cursor()

In [15]:
sql = "INSERT INTO currency(cur_date, cur_name, cur_rate) VALUES " + ",".join(result)
cur.execute(sql)
con.close()

### 원자재(석유, 금)

- API가 2020년부터 데이터가 있어서 사용이 불가능.. 하하
- Naver를 통해 크롤링해야할 듯.
  - 주식, 코스닥 코스피도 동일
- 석유 : 경유, 휘발유, 등유 3가지 평균값 활용
  - 네이버 증권에는 등유가 없어서 경유, 휘발유만 사용?
- 금 : 국내금, 국제금 2가지 각각 활용

##### api 함수

##### 오일
- 휘발유 : OIL_GSL
- 경유 : OIL_LO
- 예시 : https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=640

In [20]:
def oil_api(oil_type):
  page = 640
  HOST = "https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd={}&page=".format(oil_type)
  
  url = HOST+str(page)
  headers = {"Content-Type" : "application/json", "charset" : "UTF-8", "Accept":"*/*"}

  try:
    response = requests.get(url, headers=headers)
    return response
  except Exception as ex:
    print(ex)
  

In [24]:
page = oil_api("OIL_GSL")
soup = bs(page.text, "html.parser")

In [25]:
soup

<html lang="ko">
<head>
<title>네이버 증권</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/css/finance.css" rel="stylesheet" type="text/css"/>
<script language="javascript">document.domain="naver.com";</script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript"></script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/js/lcslog.js" type="text/javascript"></script>
</head>
<body>
<div class="section_exchange2">
<h3 class="h_today"><span>일별 시세</span></h3>
<table border="1" class="tbl_exchange today" summary="일별 시세 리스트">
<caption>일별 시세</caption>
<colgroup>
<col width="85"/>
<col width="83"/>
<col width="83"/>
<col width="82"/>
</colgroup>
<thead>
<tr>
<th class="th_today1

##### 데이터 수집

##### 저장 및 불러오기

##### DB 저장